# **Group 14 Apply AI Project: Predicting Depression through Social Media Posts**                                   
Members: Torrence Barbour, Satwik Nijampudi, Shanthan Gunti, and Ryan Edwards

Research Question: Given a sufficiently long social media post, is it possible to predict if said user has depression or not? That is what we seek to find in this project using Natural Language Processing and a supervised classification model.

In [6]:
#We need to import some libraries to do this, including some big ones like numpy, pandas, and matplotlib
import os
import time
import re
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt


import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

## Further Cleaning our data                                                                                                  
We got our data from a kaggle dataset: https://www.kaggle.com/datasets/infamouscoder/depression-reddit-cleaned and it came pre-cleaned, however, in order to show some fundamentals on Natural Language Processing, let's go over some aspects of it

In [7]:
#First, we will need to bring in our dataset. We can use numpy for this
text = pd.read_csv('depression_dataset_reddit_cleaned.csv')
text.head()

,clean_text,is_depression
0,we understand that most people who reply immed...,1
1,welcome to r depression s check in post a plac...,1
2,anyone else instead of sleeping more when depr...,1
3,i ve kind of stuffed around a lot in my life d...,1
4,sleep is my greatest and most comforting escap...,1


In [5]:
#Next, we are going to need a tool to tokenize, lemmatize, and manage our data. We will use spacy
import spacy
!python3 -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

ModuleNotFoundError: No module named 'spacy'

In [8]:
#Just to confirm that worked, lets print out one of the posts
print(text.iloc[2])

clean_text       anyone else instead of sleeping more when depr...
is_depression                                                    1
Name: 2, dtype: object


Now that we have some tools, our dataset, and another tool used for NLP installed, we can now begin. The first thing we will cover is something called tokenization. Essentially, each word (or even punctuation for that matter) in a sentence plays a role and it can be looked at as its own entity. We want to break each of these sentences apart into 'tokens' so they are more processable.

In [5]:
#Lets tokenize the sample sentence from before as an example
sample_Tokens = nlp(text.iloc[2]['clean_text'])
for part in sample_Tokens: 
    print(part.text)

anyone
else
instead
of
sleeping
more
when
depressed
stay
up
all
night
to
avoid
the
next
day
from
coming
sooner
may
be
the
social
anxiety
in
me
but
life
is
so
much
more
peaceful
when
everyone
else
is
asleep
and
not
expecting
thing
of
you


On top of tokenization, there is also lemmatization. A lemma in linguistics is essentially the core component of a word, before conjugation and the like occurs. This is very important because it essentially determines what role the word will play in the sentence. Will it be a noun? A verb? Lets see a few examples of this in code

In [6]:
#The .lemma_ command in spacy will output the lemma of the word it was called with. This will come in handy
print('The lemma of', sample_Tokens[18], 'is', sample_Tokens[18].lemma_)
print('The lemma of', sample_Tokens[4], 'is', sample_Tokens[4].lemma_)
print('The lemma of', sample_Tokens[19], 'is', sample_Tokens[19].lemma_)

The lemma of coming is come
The lemma of sleeping is sleep
The lemma of sooner is soon


In [7]:
#The .pos_ command will output the part of speech the word plays in a sentence.
print('The part of speech of the word', sample_Tokens[18].lemma_, 'is', sample_Tokens[18].pos_)
print('The part of speech of the word', sample_Tokens[19].lemma_, 'is', sample_Tokens[19].pos_)
print('The part of speech of the word', sample_Tokens[24].lemma_, 'is', sample_Tokens[24].pos_)

The part of speech of the word come is VERB
The part of speech of the word soon is ADV
The part of speech of the word anxiety is NOUN


As much as this information is very practical and useful to know, for the sake of this project and brevity we are going to be using a tool known as HuggingFace. It streamlines this process to make it more accessible to users. The information can still be used when you want extra specific cleaning of the data though.

In [3]:
import transformers as hug

With HuggingFace, we can use a model known as BERT which has been trained on lots of data in order to tokenize our sentences and turn them into word vectors in just a few commands. Without this model it could take years to do this project so suffice to say this is a must. 

In [4]:
#This is adding in the BERT tokenizer which we will use to tokenize the posts
theTokenizer = hug.AutoTokenizer.from_pretrained('bert-base-uncased')

In [5]:
#This is adding in the BERT model which we will use to get the word vectors
theModel = hug.AutoModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Since running this model requires a significant amount of computer memory (RAM), it is efficient to chop up the data into smaller pieces so that memory can be freed consistently and constantly. (Without doing this you could only get around 120 posts processed with 32GB of RAM). Note: we did drop one posts just so we could split the pieces into 1 posts each, but given the size of the dataset this should not be a concern.

In [26]:
posts = pd.read_csv('depression_dataset_reddit_cleaned.csv')
posts = posts.drop(7730)
chunked_posts = np.array_split(posts, 773)
chunked_posts

[                                          clean_text  is_depression
 0  we understand that most people who reply immed...              1
 1  welcome to r depression s check in post a plac...              1
 2  anyone else instead of sleeping more when depr...              1
 3  i ve kind of stuffed around a lot in my life d...              1
 4  sleep is my greatest and most comforting escap...              1
 5  i m year old turning soon in a few month i liv...              1
 6  i live alone and despite me being prone to lon...              1
 7  i m not looking for sympathy just simply to st...              1
 8  i don t know how to communicate all of my thou...              1
 9  mom i m sad it hurt in my heart the feeling fa...              1,
                                            clean_text  is_depression
 10  i ve been struggling with depression for a lon...              1
 11  idk how to elaborate on it i just started sudd...              1
 12  i tried to help his famil

Now lets get onto the actual conversion of posts to word vectors. By going through every post of every sub-dataframe we can add all of these embeddings and save them all to different .csv files which can be compiled into a single .csv later. Since they maintain their ordering no data is lost in this process.

In [28]:
#Parse through every post
for x in tqdm(range(len(chunked_posts))):
    word_Vectors = []
    for y in tqdm(range(len(chunked_posts[x]))):
        to_Clean = chunked_posts[x]['clean_text'][(10*x)+y]
        inputs = theTokenizer(to_Clean,truncation = True, max_length = 512, return_tensors="pt")
        output = theModel(**inputs)
    
        word_Vectors.append(output[1].detach())
        
    vectors = pd.DataFrame(word_Vectors)
    vectors.to_csv('cleaned_depression_dataset_ ' + str(x) + '.csv')
    
#Create a new data frame with the new column in it

  0%|          | 0/773 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

**And that is all for the conversion of posts to word vectors.**
From this point on, we will need to create a classification model that can map these vectors to either showing signs of depression or not.